#### Importing the libaries

In [1]:
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import numpy as np 
import cv2

In [2]:
def roi(img):
    x = int(img.shape[1])
    y = int(img.shape[0])
    shape = np.array([[int(0), int(y)], [int(x), int(y)], [int(0.55*x), int(0.6*y)], [int(0.45*x), int(0.6*y)]])

    #define a numpy array with the dimensions of img, but comprised of zeros
    mask = np.zeros_like(img)

    #Uses 3 channels or 1 channel for color depending on input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    #creates a polygon with the mask color
    cv2.fillPoly(mask, np.int32([shape]), ignore_mask_color)

    #returns the image only where the mask pixels are not zero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [3]:
def convert_to_hsv(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

In [4]:
def select_white_yellow(image):
    # Define white color range in HSV
    lower_white = np.array([0, 0, 200], dtype=np.uint8)
    upper_white = np.array([255, 30, 255], dtype=np.uint8)

    # Define yellow color range in HSV
    lower_yellow = np.array([15, 100, 100], dtype=np.uint8)
    upper_yellow = np.array([40, 255, 255], dtype=np.uint8)

    # Create masks for white and yellow colors
    white_mask = cv2.inRange(image, lower_white, upper_white)
    yellow_mask = cv2.inRange(image, lower_yellow, upper_yellow)

    # Combine masks to get lane lines
    combined_mask = cv2.bitwise_or(white_mask, yellow_mask)
    return combined_mask

In [5]:
def grayscale(img):
    """
    Converts the input image to grayscale.

    :param img: Input image
    :return: Grayscale image
    """
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [6]:
def canny(img, low_threshold, high_threshold):
    """
    Applies the Canny edge detection algorithm to the input image.

    :param img: Input image
    :param low_threshold: Lower threshold for edge detection
    :param high_threshold: Upper threshold for edge detection
    :return: Edge-detected image
    """
    return cv2.Canny(img, low_threshold, high_threshold)

In [7]:
def gaussian_blur(img, kernel_size):
    """
    Applies Gaussian blur to the input image.

    :param img: Input image
    :param kernel_size: Size of the Gaussian kernel
    :return: Blurred image
    """
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

In [8]:
def draw_lines(img, lines, color=(0, 255, 0), thickness=5):
    """
    Draws lines on the input image.

    :param img: Input image
    :param lines: List of lines to be drawn
    :param color: Line color (BGR format)
    :param thickness: Line thickness
    """
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

In [27]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    Applies the Hough line transform to detect lines in the input image.

    :param img: Input image
    :param rho: Distance resolution of the accumulator in pixels
    :param theta: Angle resolution of the accumulator in radians
    :param threshold: Minimum number of votes required to consider a line
    :param min_line_len: Minimum length of a line to be considered
    :param max_line_gap: Maximum gap between segments to be connected as a line
    :return: Image with detected lines drawn
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

In [28]:
def lane_detection_pipeline(image): 
    """
    Lane detection pipeline that applies preprocessing steps and detects lanes.

    :param image: Input image
    :return: Image with detected lanes drawn
    """
    # Define region of interest vertices 
    roi_vertices = np.array([[(100, image.shape[0]), (image.shape[1] / 2 - 50, image.shape[0] / 2 + 50),
                              (image.shape[1] / 2 + 50, image.shape[0] / 2 + 50), (image.shape[1] - 100, image.shape[0])]],
                            dtype=np.int32)
    # Convert the image to HSV color space
    hsv_image = convert_to_hsv(image)

    # Select white and yellow lane lines
    color_filtered_image = select_white_yellow(hsv_image)
    
    # Apply Gaussian blur to the grayscale image 
    blurred_image = gaussian_blur(color_filtered_image, kernel_size=5)
    
    # Apply Canny edge detection
    edges = canny(blurred_image, low_threshold=50, high_threshold=150)
    
    # Apply region of interest mask
    masked_edges = roi(edges)
    
    # Apply Hough transform to detect lines
    line_image = hough_lines(masked_edges, rho=1, theta=np.pi/180, threshold=40, min_line_len=60, max_line_gap=20)
    
    # Combine the original image with the detected lines 
    result = cv2.addWeighted(image, 0.8, line_image, 1, 0)
    return result

In [29]:
def process_image(image_path): 
    """
    Process a single image using the lane detection pipeline.

    :param image_path: Path to the input image
    :return: Image with detected lanes drawn
    """
    img = cv2.imread(image_path)
    result = lane_detection_pipeline(img)
    return result

In [30]:
def process_video(input_path, output_path):
    """
    Process a video using the lane detection pipeline and save the output.

    :param input_path: Path to the input video
    :param output_path: Path to save the output video
    """
    video_capture = cv2.VideoCapture(input_path)
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))
    frame_size = (int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)), 
                 int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, fps, frame_size)
    
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
        
        result = lane_detection_pipeline(frame)
        out.write(result)

    video_capture.release()
    out.release()
    cv2.destroyAllWindows()

In [ ]:
# Process an image
image_result = process_image('./img/exit-ramp.jpg')
cv2.imshow('Lane Detection Image', image_result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [32]:
# Process a video
process_video('./vid/challenge.mp4', 'output_video1.mp4')